# Detect (with the) Model
## Step 1 - run the model locally

In this notebook, we are replicating  some of the original MobileNet TensorFlow Lite functionality.  That means taking the TensorFlow model (checkpoint) and converting it to a TensorFlow Lite model.   This is not quite what the SageMaker tutorial does.   The tutorial sticks with TensorFlow.   Confirm that the Lite model works first - because that is a known path then:
- Detect Step 2 - run endpoint as a TensorFlow model
- Detect Step 3 - run endpoint as a TensorFlow Lite Model -- the advantage of a Lite model is less latency, faster, less resources -- with slightly less accuracy.    Lite models are the path towards IoT deployments and TPU compatible models

We are testing the model before we deploy it as an endpoint.   This is optional but definitely makes sense as we learn this process

In [ ]:
import os

## Global Environment Variables

We are using objects and scripts in the project as much as possible.   They require environment variables to pass along where stuff is stored

In [ ]:
PROJECT_DIR = os.getcwd()
IMAGE_DIR = os.path.join(PROJECT_DIR, "data/new_jpeg_immages")
MODEL_NAME = 'tf_lite'   # not using the edge_tpu 
MODEL_PATH = os.path.join(PROJECT_DIR, "tflite_model/output_tflite_graph.tflite")
LABEL_MAP = os.path.join(PROJECT_DIR, "code/cfa_prod_label_map.pbtxt")
DISPLAY = "None"
ANNOTATION_DIR = os.path.join(PROJECT_DIR, "data/unverified_annotations")
S3_TEST_IMAGES = "s3://cfaanalyticsresearch-sagemaker/datasets/cfa_products/test_images/"

## Data
Copy the test images locally

In [ ]:
! aws s3 cp {S3_TEST_IMAGES} {IMAGE_DIR} --recursive

## Detect
The detect.py script will read a directory of images, infer and generate XML (unverified) Annotations (VOC PASCAL schema).    Unverified means if you run them through a labeling program like labelImg, they do not have the verified attribute.    (Downstream, they will be ignored if you try to fold them into training data.)

In [ ]:
! python code/detect.py --image_dir {IMAGE_DIR} --model_name {MODEL_NAME} --model_path {MODEL_PATH} --label_map_path {LABEL_MAP} --display {DISPLAY} --annotation_dir {ANNOTATION_DIR}

In [ ]:
# tarball the annotations
os.chdir("data")
! tar -czvf  unverif_annotations.tar.gz unverified_annotations

## Conclusion

If you want the annotations - you'll find the annotations tarball in data/ directory
use the Notebook browser to download it

The fastest, easiest way to review (and correct / verify) is to use labelImg program which will merge the image and annotation

The main conclusion here is our model works.